In [28]:
import json
with open('ald_meta.json', 'r', encoding='utf-8') as f:
    ald_meta = json.load(f)
with open('ald_sen.json', 'r', encoding='utf-8') as f:
    ald_sen = json.load(f)

In [31]:
print(len(ald_meta.keys()))

35070


In [4]:
count = 0
for m in ald_meta.keys():
    if 'year' not in ald_meta[m].keys():
        count += 1
print(count)

91


In [10]:
from chemdataextractor import Document
from tqdm import tqdm

# copy and paste abstract text into Document
with open('ald.json', 'r', encoding='utf-8') as f:
    ald_dict = json.load(f)
spans = {}
for a in tqdm(ald_dict.keys()):
    doc = Document(ald_dict[a])
    spans[a] = doc.cems

100%|██████████| 34752/34752 [26:17<00:00, 22.03it/s]


In [11]:
def find_nearest(sen, ci, span):
    distance = 10000
    nearest = ''
    # print(span)
    for s in span:
        cd = abs(s.start-ci)
        if cd < distance:
            distance = cd
            nearest = s.text
    return nearest

In [12]:
mat_freq = {}
o_record = {}
with open('ald.txt', 'r', encoding='utf-8') as f:
    data = f.readlines()
    for sen in tqdm(data):
        o_record[sen[:-1]] = {}
        doi = ald_sen[sen[:-1]]['doi']
        if 'year' in ald_meta[doi].keys():
            time = ald_meta[doi]['year']
            o_record[sen[:-1]]['time'] = time
        else:
            time = '?'
            print('?')
            o_record[sen[:-1]]['time'] = time
        if 'i_d' in ald_meta[doi].keys():
            id_ = ald_meta[doi]['i_d']  
            o_record[sen[:-1]]['id'] = id_
        else:
            id_ = '?'  
            o_record[sen[:-1]]['id'] = id_
        doc = Document(sen[:-1])
        if len(doc.records) == 0:
            current_index = ald_sen[sen[:-1]]['index']
            span = spans[doi]
            nearest_m = find_nearest(sen[:-1], current_index, span)
            r_list = [nearest_m]
        else:
            r_list = []
            for r in doc.records:
                r_d = r.serialize()
                r_list.append(r_d['names'][0])
        o_record[sen[:-1]]['mat'] = r_list
        for n in r_list:
            if n in mat_freq.keys():
                mat_freq[n]['freq'] += 1
                if time in mat_freq[n]['year_freq'].keys():
                    mat_freq[n]['year_freq'][time] += 1
                else:
                    mat_freq[n]['year_freq'][time] = 1
            else:
                mat_freq[n] = {}
                mat_freq[n]['freq'] = 1
                mat_freq[n]['year_freq'] = {}
                mat_freq[n]['year_freq'][time] = 1

  2%|▏         | 49/2190 [00:05<03:56,  9.07it/s]

?


 15%|█▍        | 325/2190 [00:37<03:38,  8.55it/s]

?


 15%|█▍        | 326/2190 [00:37<04:42,  6.60it/s]

...

 93%|█████████▎| 2036/2190 [04:09<00:19,  7.91it/s]

?


 94%|█████████▍| 2056/2190 [04:11<00:10, 12.73it/s]

?


100%|██████████| 2190/2190 [04:29<00:00,  8.12it/s]


In [15]:
import kashgari
from kashgari.tasks.classification import CNN_Attention_Model, BiLSTM_Model, BiGRU_Model, CNN_Model, CNN_LSTM_Model
import json
from collections import Counter

loaded_model = CNN_Attention_Model.load_model('CNN_Attention_86')

2022-02-19 14:30:51,280 [DEBUG] kashgari - ------------------------------------------------
2022-02-19 14:30:51,281 [DEBUG] kashgari - Loaded gensim word2vec model's vocab
2022-02-19 14:30:51,282 [DEBUG] kashgari - model        : mat_embedding3/vectors.txt
2022-02-19 14:30:51,282 [DEBUG] kashgari - word count   : 529690
2022-02-19 14:30:51,283 [DEBUG] kashgari - Top 50 words : ['the', 'of', '.', ',', 'and', '<nUm>', 'in', 'a', 'to', ')', '(', 'with', '-', 'for', 'is', 'by', 'on', 'was', 'at', 'were', 'that', '–', 'as', 'are', 'from', '/', 'an', 'temperature', 'surface', 'using', 'high', 'which', 'C', '°', 'this', '%', 'In', 'it', 'A', '=', 'structure', 'properties', ':', 'phase', 'results', 'effect', 'these', 'than', 'based', 'different']
2022-02-19 14:30:51,283 [DEBUG] kashgari - ------------------------------------------------


In [24]:
for_predict = []
for o in o_record.keys():
    for_predict.append(o.split(' '))
results = loaded_model.predict(for_predict)
for i, o in enumerate(o_record.keys()):
    o_record[o]['attitude'] = results[i]

2022-02-19 15:19:56,495 [DEBUG] kashgari - predict input shape (2187, 78) x: 
[[   0    1  462 ...    0    0    0]
 [   0    1 2770 ...    0    0    0]
 [   0    1   50 ...    0    0    0]
 ...
 [   0    1 3534 ...    0    0    0]
 [   0    1  344 ...    0    0    0]
 [   0    1    7 ...    0    0    0]]
2022-02-19 15:19:57,618 [DEBUG] kashgari - predict output shape (2187, 2)
2022-02-19 15:19:57,620 [DEBUG] kashgari - predict output argmax: [0 0 1 ... 0 0 1]


In [30]:
count = 0
for t in results:
    if t == 'driver':
        count += 1
print(count)

1690


In [25]:
json_str = json.dumps(o_record, indent=4)
with open('ald_sen_record.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_str)
json_str = json.dumps(mat_freq, indent=4)
with open('ald_mat_record.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_str)

In [26]:
import json

with open('ald_sen_record.json', 'r', encoding='utf-8') as f:
    ald_sen_record = json.load(f)
    
for s in ald_sen_record.keys():
    ald_sen_record[s]['doi'] = ald_sen[s]['doi']
    
json_str = json.dumps(ald_sen_record, indent=4)
with open('ald_sen_record.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_str)

In [7]:
import json
with open('ald_sen_record.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
with open('ald_mat_record.json', 'r', encoding='utf-8') as f:
    mdata = json.load(f)
print(mdata['fullerene'])
for d in data.keys():
    if 'fullerene' in data[d]['mat']:
        print(d, data[d]['time'])
print('graphene', ':', mdata['graphene'])



{'freq': 2, 'year_freq': {'2018': 1, '2007': 1}}
Compositional tuning of the perovskite material can improve the fullerene-free device performance . 2018
Such a close relationship between the structure and photophysical and photoelectrochernical properties of the fullerene dendrimers and their nanoclusters will provide knowledge of photophysics regarding photoactive molecular assemblies with dendritic architectures . 2007
graphene : {'freq': 78, 'year_freq': {'2018': 17, '2013': 3, '2020': 6, '2014': 5, '2015': 5, '2019': 9, '2021': 9, '2016': 6, '2017': 12, '2012': 2, '2010': 1, '?': 1, '2022': 2}}
